# Impyte Documentation
This is a first practicle attempt to clarify the usage of `impyte`. It's a collection of easy applicable and reproducible examples that you could use to simplyify your data processing workflow.

## Importing and generating sample data
In order to show some of the features of the library, we'll be using Kaggle's HR data that can be found [here](https://www.kaggle.com/ludobenistant/hr-analytics).

In [1]:
# import library and data set
from importlib import reload
import impyte3
reload(impyte3)
import pandas as pd
from tools.data_prep import remove_random

In [2]:
data = pd.read_csv('data/hr_test.csv')

In [3]:
data.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low


## Add random missing values
In order to show some of the pattern visualization and imputation methods, we need incomplete data sets. To achieve this, we'll be using a helper function that deletes values at random. 25 % of all values from each category will be deleted.

In [4]:
data = remove_random(data, .25, randomize_index=False)

In [5]:
data.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,NaN,0.79,5.0,171.0,5.0,0.0,0.0,0.0,IT,medium
1,NaN,0.92,3.0,260.0,2.0,0.0,0.0,0.0,sales,low
2,NaN,0.73,3.0,252.0,4.0,0.0,0.0,0.0,product_mng,high
3,NaN,0.49,5.0,137.0,3.0,0.0,0.0,0.0,hr,medium
4,NaN,0.55,5.0,204.0,3.0,0.0,0.0,0.0,support,low


In [6]:
data.describe()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years
count,14625.000000,14625.000000,14625.000000,14625.000000,14625.000000,14625.000000,14625.000000,14625.000000
mean,0.612746,0.716209,3.801846,200.944342,3.498051,0.144547,0.238291,0.021402
std,0.248620,0.171056,1.230642,49.895518,1.459430,0.351656,0.426052,0.144724
min,0.090000,0.360000,2.000000,96.000000,2.000000,0.000000,0.000000,0.000000
25%,0.440000,0.560000,3.000000,156.000000,3.000000,0.000000,0.000000,0.000000
50%,0.640000,0.720000,4.000000,200.000000,3.000000,0.000000,0.000000,0.000000
75%,0.820000,0.870000,5.000000,245.000000,4.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,7.000000,310.000000,10.000000,1.000000,1.000000,1.000000


In [7]:
len(data)

14999

In [8]:
reload(impyte3)

<module 'impyte3' from '/Users/andirs/Dropbox (Personal)/_Studium/04_Semester/Projektarbeit/impyter/impyte3.py'>

## Testing rudimentary features
Below is a first trial run of some of the features implemented by impyter and its helper classes.

## `NanChecker`
Functionality testing of `NanChecker` class.

In [9]:
nan_checker = impyte3.NanChecker()

#### `NanChecker.is_nan(data, nan_vals=None, recursive=True)`
Detect missing values (NaN in numeric arrays, empty strings in string arrays). NaN detection feature also enables NaN-value ingestion as parameter values.

In [10]:
# [True, False, False]
print(nan_checker.is_nan(["", 'None', 'NaN']))

# [True, True, True]
print(nan_checker.is_nan(["", 'None', 'NaN'], nan_vals=['', None, 'None', 'NaN']))

[True, False, False]
[True, True, True]


In [11]:
# Recursive nan detection
# [True, True, False, [False, True, True]]
print(nan_checker.is_nan(["", None, 'NaN', ["List Value 1", '', None]]))

[True, True, False, [False, True, True]]


In [12]:
# Values can be declared as nan-values
# [True, False, False, True]
nan_checker.is_nan(['NaN', 'Empty', 'None', 'N/A'], nan_vals=['NaN', 'N/A'])

[True, False, False, True]

## `Pattern`
Functionality testing of `Pattern` class. The `Pattern` class stores different patterns and data summaries regarding NaN values. 

In [13]:
reload(impyte3)
pattern_log = impyte3.Pattern()

#### `Pattern._check_complete_row(row)`
Determines whether a row consists out of only 1s. Method for NaN summary creation

In [14]:
pd.DataFrame([1, 1, 1, 'NaN']).apply(pattern_log._check_complete_row)
#pattern_log._check_complete_row()

0    0
dtype: int64

#### `Pattern._compute_pattern(data, nan_values="", verbose=False)`
Function that checks for missing values and prints out a quick table of a summary of missing values. Includes pattern overview and counts of missing values by column. Computes pattern and returns its pattern structure plus the count of data points for each of these patterns. To get a nice digestible table add `["table"]` selector to output.

In [15]:
pattern_dict = pattern_log._compute_pattern(data)
indices, table = pattern_dict["indices"], pattern_dict["table"]

In [16]:
table

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary,Count
0,1,1,1,1,1,1,1,1,1,1,11259
1,NaN,1,1,1,1,1,1,1,1,1,374
2,1,NaN,1,1,1,1,1,1,1,1,374
3,1,1,NaN,1,1,1,1,1,1,1,374
4,1,1,1,NaN,1,1,1,1,1,1,374
5,1,1,1,1,NaN,1,1,1,1,1,374
6,1,1,1,1,1,NaN,1,1,1,1,374
7,1,1,1,1,1,1,NaN,1,1,1,374
8,1,1,1,1,1,1,1,NaN,1,1,374
9,1,1,1,1,1,1,1,1,NaN,1,374


In [17]:
indices[0][:10] # first 10 indices of pattern 0

[374, 749, 1124, 1499, 1874, 2249, 2624, 2999, 3374, 3749]

In [18]:
pattern_nr = 0
print("Pattern {} has {:,} rows.".format(pattern_nr, len(indices[0])))

Pattern 0 has 11,259 rows.


#### `Pattern._is_discrete(tmpdata, unique_instances)`
Determines on dtype and by counting unique instances whether a column contains categorical or continuous values.

In [19]:
pattern_log._is_discrete(data['satisfaction_level'], unique_instances=5) # False

False

In [20]:
pattern_log._is_discrete(data['sales'], unique_instances=5) # True

True

#### `Pattern._get_discrete_and_continuous(tmpdata, unique_instances)`
Returns the column names of discrete and continuous variables. Column names are stored in lists for easy selection. Those lists are stored in one dictionary object. All continuous column names can be accessed through `['continuous']` all discrete variables through `['discrete']`.

In [21]:
pattern_log._get_discrete_and_continuous(data, unique_instances=5)

{'continuous': ['satisfaction_level',
  'last_evaluation',
  'number_project',
  'average_montly_hours',
  'time_spend_company'],
 'discrete': ['Work_accident',
  'left',
  'promotion_last_5years',
  'sales',
  'salary']}

#### `Pattern._get_missing_value_percentage(self, data, importance_filter=False)`
Shows missing value percentage and count of unique values in category based on result and actual data table.

In [22]:
pattern_log.get_missing_value_percentage(data)

,Complete,Missing,Percentage,Unique
satisfaction_level,14625,374,2.49 %,93
last_evaluation,14625,374,2.49 %,66
number_project,14625,374,2.49 %,7
average_montly_hours,14625,374,2.49 %,216
time_spend_company,14625,374,2.49 %,9
Work_accident,14625,374,2.49 %,3
left,14625,374,2.49 %,3
promotion_last_5years,14625,374,2.49 %,3
sales,14625,374,2.49 %,11
salary,14625,374,2.49 %,4


#### `Pattern.get_pattern()`
Returns NaN-patterns based on primary computation or initiates new computation of NaN-patterns. Uses `_compute_patterns()` to visualize patterns if not yet computed. Otherwise returns stored information regarding patterns.

In [23]:
pattern_log.get_pattern()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary,Count
0,1,1,1,1,1,1,1,1,1,1,11259
1,NaN,1,1,1,1,1,1,1,1,1,374
2,1,NaN,1,1,1,1,1,1,1,1,374
3,1,1,NaN,1,1,1,1,1,1,1,374
4,1,1,1,NaN,1,1,1,1,1,1,374
5,1,1,1,1,NaN,1,1,1,1,1,374
6,1,1,1,1,1,NaN,1,1,1,1,374
7,1,1,1,1,1,1,NaN,1,1,1,374
8,1,1,1,1,1,1,1,NaN,1,1,374
9,1,1,1,1,1,1,1,1,NaN,1,374


#### `Pattern.row_nan_pattern(row)`
This is a core piece to determining which patterns are available. Other methods work with the output of `row_nan_pattern()` to determine unique pattern structures and count these before turning them into readable tables.

In [24]:
# (1, 1, 'NaN', 'NaN')
print(pattern_log.row_nan_pattern(['Value 1', 'Value 2', '', None]))

(1, 1, 'NaN', 'NaN')


#### `Pattern.get_pattern_indices(pattern_no)`

In [25]:
pattern_log.get_pattern_indices(0)[:10] # get first 10 indices of pattern 0

[374, 749, 1124, 1499, 1874, 2249, 2624, 2999, 3374, 3749]

#### `Pattern.get_continuous()`
Returns list with names of all continuous variables.

In [26]:
pattern_log.get_continuous()

['satisfaction_level', 'last_evaluation', 'average_montly_hours']

#### `Pattern.get_discrete()`
Returns list with names of all discrete variables.

In [27]:
pattern_log.get_discrete()

['number_project',
 'time_spend_company',
 'Work_accident',
 'left',
 'promotion_last_5years',
 'sales',
 'salary']

## `Impyter`

In [28]:
reload(impyte3)
imp = impyte3.Impyter() # instantiate impyte class

### Load data into imputer

#### `Impyter.load_data()`
Requires a pandas DataFrame to load. Otherwise, the input is being transformed into a DataFrame. While loading the data is being copied into the object, to stay clear of consistency issues with the original data set.

In [29]:
imp.load_data(data)

As an alternative a DataFrame can be handed over while instantiating the Impyter object.

In [30]:
imp = impyte3.Impyter(data)

#### `Impyter.get_data()`
Returns the loaded data for quick reference.

In [31]:
imp.get_data().head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,NaN,0.79,5.0,171.0,5.0,0.0,0.0,0.0,IT,medium
1,NaN,0.92,3.0,260.0,2.0,0.0,0.0,0.0,sales,low
2,NaN,0.73,3.0,252.0,4.0,0.0,0.0,0.0,product_mng,high
3,NaN,0.49,5.0,137.0,3.0,0.0,0.0,0.0,hr,medium
4,NaN,0.55,5.0,204.0,3.0,0.0,0.0,0.0,support,low


#### `Impyter.pattern()`
Leverages `Pattern._compute_pattern()` and `Pattern.get_pattern()` methods to compute and return an overview of all existant NaN patterns in the data set. The overview shows a `NaN` in the column where a data point was missing and `1` for all complete slots. On the right hand side is a count variable to indicate how often that pattern was found. The patterns are always sorted by count and it is not given, that pattern 0 is always the pattern with only complete cases.

In [32]:
reload(impyte3)
imp = impyte3.Impyter(data) # instantiate impyte class
nan_checker = impyte3.NanChecker()
pattern_log = impyte3.Pattern()
imp.pattern()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary,Count
0,1,1,1,1,1,1,1,1,1,1,11259
1,NaN,1,1,1,1,1,1,1,1,1,374
2,1,NaN,1,1,1,1,1,1,1,1,374
3,1,1,NaN,1,1,1,1,1,1,1,374
4,1,1,1,NaN,1,1,1,1,1,1,374
5,1,1,1,1,NaN,1,1,1,1,1,374
6,1,1,1,1,1,NaN,1,1,1,1,374
7,1,1,1,1,1,1,NaN,1,1,1,374
8,1,1,1,1,1,1,1,NaN,1,1,374
9,1,1,1,1,1,1,1,1,NaN,1,374


#### `Impyter.drop_pattern()`
Drops pattern from data set.

In [33]:
imp.drop_pattern(4).head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,NaN,0.79,5.0,171.0,5.0,0.0,0.0,0.0,IT,medium
1,NaN,0.92,3.0,260.0,2.0,0.0,0.0,0.0,sales,low
2,NaN,0.73,3.0,252.0,4.0,0.0,0.0,0.0,product_mng,high
3,NaN,0.49,5.0,137.0,3.0,0.0,0.0,0.0,hr,medium
4,NaN,0.55,5.0,204.0,3.0,0.0,0.0,0.0,support,low


In [34]:
imp.pattern()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary,Count
0,1,1,1,1,1,1,1,1,1,1,11259
1,NaN,1,1,1,1,1,1,1,1,1,374
2,1,NaN,1,1,1,1,1,1,1,1,374
3,1,1,NaN,1,1,1,1,1,1,1,374
4,1,1,1,NaN,1,1,1,1,1,1,374
5,1,1,1,1,NaN,1,1,1,1,1,374
6,1,1,1,1,1,NaN,1,1,1,1,374
7,1,1,1,1,1,1,NaN,1,1,1,374
8,1,1,1,1,1,1,1,NaN,1,1,374
9,1,1,1,1,1,1,1,1,NaN,1,374


If inplace flag set to True, the changes happen in the data set that's being stored in the Impyte object. Otherwise, a copy without the dropped pattern will be returned and the stored data set stays intact.

In [35]:
_ = imp.drop_pattern(4, inplace=True)

In [36]:
imp.pattern()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary,Count
0,1,1,1,1,1,1,1,1,1,1,11259
1,NaN,1,1,1,1,1,1,1,1,1,374
2,1,NaN,1,1,1,1,1,1,1,1,374
3,1,1,NaN,1,1,1,1,1,1,1,374
5,1,1,1,1,NaN,1,1,1,1,1,374
6,1,1,1,1,1,NaN,1,1,1,1,374
7,1,1,1,1,1,1,NaN,1,1,1,374
8,1,1,1,1,1,1,1,NaN,1,1,374
9,1,1,1,1,1,1,1,1,NaN,1,374
10,1,1,1,1,1,1,1,1,1,NaN,374


#### `Impyter.get_summary()`
Returns table with information on missing values per column, its percentage and the count of unique values within that column.

In [37]:
imp.get_summary()

,Complete,Missing,Percentage,Unique
satisfaction_level,14251,374,2.56 %,93
last_evaluation,14251,374,2.56 %,66
number_project,14251,374,2.56 %,7
time_spend_company,14251,374,2.56 %,9
Work_accident,14251,374,2.56 %,3
left,14251,374,2.56 %,3
promotion_last_5years,14251,374,2.56 %,3
sales,14251,374,2.56 %,11
salary,14251,374,2.56 %,4


In [38]:
data['Work_accident'].unique()

array([  0.,   1.,  nan])

Setting the importance filter flag to `True`  shows only columns that have some missing values. This is helpful for data sets with a large amount of variables and only few nan-values.

In [39]:
for pattern_no in range(1,4): #  drop patterns 1 to 3
    imp.drop_pattern(pattern_no, inplace=True)

In [40]:
imp.get_summary(importance_filter=True)

,Complete,Missing,Percentage,Unique
time_spend_company,13129,374,2.77 %,9
Work_accident,13129,374,2.77 %,3
left,13129,374,2.77 %,3
promotion_last_5years,13129,374,2.77 %,3
sales,13129,374,2.77 %,11
salary,13129,374,2.77 %,4


#### `Impyte.one_hot_encode()`
Relies on `pandas.get_dummies()` method to transform categorical values into one-hot-encoded values.

In [41]:
reload(impyte3)
imputer = impyte3.Impyter()
imputer.load_data(data)
_ = imputer.pattern()

In [42]:
ohe_data = imputer.one_hot_encode(data)
ohe_data.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales_ohe_IT,sales_ohe_RandD,...,sales_ohe_hr,sales_ohe_management,sales_ohe_marketing,sales_ohe_product_mng,sales_ohe_sales,sales_ohe_support,sales_ohe_technical,salary_ohe_high,salary_ohe_low,salary_ohe_medium
0,NaN,0.92,3.0,206.0,3.0,0.0,0.0,0.0,0,1,...,0,0,0,0,0,0,0,0,0,1
1,NaN,0.37,3.0,108.0,4.0,0.0,0.0,0.0,0,0,...,0,0,0,0,0,0,1,0,1,0
2,NaN,0.52,4.0,184.0,4.0,0.0,0.0,0.0,0,0,...,0,0,0,0,1,0,0,0,1,0
3,NaN,0.79,5.0,271.0,4.0,0.0,0.0,0.0,0,0,...,0,1,0,0,0,0,0,1,0,0
4,NaN,0.51,3.0,241.0,6.0,1.0,0.0,0.0,0,0,...,0,0,0,0,0,1,0,0,1,0


#### `Impyte.one_hot_decode()`
The inversion method to `Impyte.one_hot_encode()`. Transforms one-hot-encoded columns back to categorical values.

In [43]:
imputer.one_hot_decode(ohe_data).head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,NaN,0.92,3.0,206.0,3.0,0.0,0.0,0.0,RandD,medium
1,NaN,0.37,3.0,108.0,4.0,0.0,0.0,0.0,technical,low
2,NaN,0.52,4.0,184.0,4.0,0.0,0.0,0.0,sales,low
3,NaN,0.79,5.0,271.0,4.0,0.0,0.0,0.0,management,high
4,NaN,0.51,3.0,241.0,6.0,1.0,0.0,0.0,support,low


#### `Impyter.impute()`
Impute is the core method of impyte. The method works out of the box and uses Random Forest estimators per default to impute missing values. It automatically performs cross-validation to showcase the potential accuracy of the imputation. Scoring that is being used is f1_macro score for classifiers (supporting binary and multi-class) and r2 for regression models.

In [71]:
reload(impyte3)
imputer = impyte3.Impyter()
imputer.load_data(data)

In [72]:
_ = imputer.pattern()
complete_df = imputer.impute(estimator='rf')

Scoring Threshold             Classification                Regression                    
                              None                          None                          

Pattern: Label                Score                         Estimator                     
1: satisfaction_level         0.459 (r2)                    RandomForestRegressor           imputed...
2: last_evaluation            0.298 (r2)                    RandomForestRegressor           imputed...
3: number_project             0.571 (f1_macro)              RandomForestClassifier          imputed...
4: average_montly_hours       0.324 (r2)                    RandomForestRegressor           imputed...
5: time_spend_company         0.566 (f1_macro)              RandomForestClassifier          imputed...
6: Work_accident              0.601 (f1_macro)              RandomForestClassifier          imputed...
7: left                       0.982 (f1_macro)              RandomForestClassifier          imputed...
8: pr

In [63]:
imputer.impute()

Scoring Threshold             Classification                Regression                    
                              None                          None                          

Pattern: Label                Score                         Estimator                     


,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,NaN,0.79,5.0,171.0,5.0,0.0,0.0,0.0,IT,medium
1,NaN,0.92,3.0,260.0,2.0,0.0,0.0,0.0,sales,low
2,NaN,0.73,3.0,252.0,4.0,0.0,0.0,0.0,product_mng,high
3,NaN,0.49,5.0,137.0,3.0,0.0,0.0,0.0,hr,medium
4,NaN,0.55,5.0,204.0,3.0,0.0,0.0,0.0,support,low
5,NaN,0.70,5.0,135.0,2.0,0.0,0.0,1.0,accounting,medium
6,NaN,0.52,2.0,102.0,6.0,0.0,0.0,0.0,technical,low
7,NaN,0.89,5.0,179.0,2.0,0.0,0.0,0.0,technical,low
8,NaN,0.41,3.0,167.0,10.0,0.0,0.0,0.0,support,low
9,NaN,0.93,4.0,161.0,3.0,0.0,0.0,0.0,sales,low


##### Investigate completed data set

In [64]:
imp = impyte3.Impyter(complete_df)

In [65]:
imp.pattern()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary,Count
0,1,1,1,1,1,1,1,1,1,1,14999


##### Scoring measures
In order to fill in only columns that surpass a certain scoring threshold (i.e. f1 score > .7), the threshold parameter can be set. The first value determines classification (f1) the second one regression (r2).

In [75]:
complete_df_threshold = imputer.impute(estimator='rf', threshold=[.7, .5])

Scoring Threshold             Classification                Regression                    
                              0.7                           0.5                           

Pattern: Label                Score                         Estimator                     
1: satisfaction_level         0.457 (r2)                    RandomForestRegressor           not imputed...
2: last_evaluation            0.300 (r2)                    RandomForestRegressor           not imputed...
3: number_project             0.579 (f1_macro)              RandomForestClassifier          not imputed...
4: average_montly_hours       0.331 (r2)                    RandomForestRegressor           not imputed...
5: time_spend_company         0.575 (f1_macro)              RandomForestClassifier          not imputed...
6: Work_accident              0.601 (f1_macro)              RandomForestClassifier          not imputed...
7: left                       0.977 (f1_macro)              RandomForestClassifier  

In [76]:
imp = impyte3.Impyter(complete_df_threshold)
imp.pattern()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary,Count
0,1,1,1,1,1,1,1,1,1,1,12007
1,NaN,1,1,1,1,1,1,1,1,1,374
2,1,NaN,1,1,1,1,1,1,1,1,374
3,1,1,NaN,1,1,1,1,1,1,1,374
4,1,1,1,NaN,1,1,1,1,1,1,374
5,1,1,1,1,NaN,1,1,1,1,1,374
6,1,1,1,1,1,NaN,1,1,1,1,374
7,1,1,1,1,1,1,1,1,NaN,1,374
8,1,1,1,1,1,1,1,1,1,NaN,374


##### Multi-Nans
Prediction of values with multi-nan is a last resort option. This might be suitable for certain edge cases but if the score values are low it should be considered dropping the feature or the data points.

In [73]:
reload(impyte3)
multi_data = data.copy()
import numpy as np
for i in range(0, 100):
    multi_data.at[i, "last_evaluation"] = np.nan
reload(impyte3)
imp = impyte3.Impyter(multi_data)
_ = imp.pattern()
imp.pattern_log.get_multi_nan_pattern_nos()
imp.pattern_log.get_column_name(11)
res = imp.impute(estimator="rf", multi_nans=True)

Scoring Threshold             Classification                Regression                    
                              None                          None                          

Pattern: Label                Score                         Estimator                     
1: last_evaluation            0.297 (r2)                    RandomForestRegressor           imputed...
2: number_project             0.573 (f1_macro)              RandomForestClassifier          imputed...
3: average_montly_hours       0.330 (r2)                    RandomForestRegressor           imputed...
4: time_spend_company         0.567 (f1_macro)              RandomForestClassifier          imputed...
5: Work_accident              0.595 (f1_macro)              RandomForestClassifier          imputed...
6: left                       0.981 (f1_macro)              RandomForestClassifier          imputed...
7: promotion_last_5years      0.785 (f1_macro)              RandomForestClassifier          imputed...
8: sa

In [206]:
reload(impyte3)
imp.drop_imputation([.5, .7])

Dropping pattern 1 (0.2842536101354798 < 0.7 r2)
Dropping pattern 3 (0.32252753146648017 < 0.7 r2)
Dropping pattern 8 (0.33441272501906194 < 0.5 f1_macro)
Dropping pattern 10 (0.4486665293558607 < 0.7 r2)
Dropping pattern 11 (0.4098585852873728 < 0.7 r2)


In [208]:
imp.pattern() # this needs to be updated as well (?)

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary,Count
0,1,1,1,1,1,1,1,1,1,1,11259
1,1,NaN,1,1,1,1,1,1,1,1,374
2,1,1,NaN,1,1,1,1,1,1,1,374
3,1,1,1,NaN,1,1,1,1,1,1,374
4,1,1,1,1,NaN,1,1,1,1,1,374
5,1,1,1,1,1,NaN,1,1,1,1,374
6,1,1,1,1,1,1,NaN,1,1,1,374
7,1,1,1,1,1,1,1,NaN,1,1,374
8,1,1,1,1,1,1,1,1,NaN,1,374
9,1,1,1,1,1,1,1,1,1,NaN,374


In [69]:
mdl = imp.get_model(11)

In [70]:
mdl.get_model()

[RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
            max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0, warm_start=False)]

In the case of multi-nan, `drop_imputation` will average the score of all models. Yet, performing this method is discouraged. Further individual treatment of the data set might be needed in order to preprocess the information correctly. One potential action could be, to drop multi-nan columns if they contain no information.

In [176]:
np.mean(.2)

0.20000000000000001

#### `Impyter.get_result()`
Returns the results data set, once impute was performed.

In [79]:
imputer.get_result().head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,NaN,0.81,3.0,168.0,4.0,0.0,0.0,0.0,support,low
1,NaN,0.63,3.0,250.0,4.0,0.0,0.0,0.0,IT,low
2,NaN,0.76,4.0,133.0,2.0,0.0,0.0,0.0,accounting,low
3,NaN,0.65,3.0,153.0,2.0,0.0,0.0,0.0,marketing,medium
4,NaN,0.97,3.0,257.0,3.0,0.0,0.0,0.0,marketing,low


#### `Impyter.get_model()`

In [80]:
model = imp.get_model(11)

In [81]:
model.feature_name[0]

'satisfaction_level'

In [82]:
model.estimator_name[0]

'RandomForestRegressor'

In [83]:
model.scoring[0]

'r2'

In [86]:
model.threshold[0]

array([ 0.40774179,  0.41508382,  0.40658246,  0.44796795,  0.43112091])

In [87]:
model.model[0]

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [88]:
imp2 = impyte3.Impyter(res)
imp2.pattern()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary,Count
0,1,1,1,1,1,1,1,1,1,1,14999


In [89]:
imp2.get_data().head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,0.454,0.701,3.0,168.0,4.0,0.0,0.0,0.0,support,low
1,0.585,0.829,3.0,250.0,4.0,0.0,0.0,0.0,IT,low
2,0.451,0.816,4.0,133.0,2.0,0.0,0.0,0.0,accounting,low
3,0.375,0.860,3.0,153.0,2.0,0.0,0.0,0.0,marketing,medium
4,0.510,0.803,3.0,257.0,3.0,0.0,0.0,0.0,marketing,low


### Comparison of complete indices methods

In [90]:
%timeit len(imputer.pattern_log.get_complete_indices())

888 µs ± 52.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [91]:
_ = imputer.drop_pattern(7, inplace=True)

In [92]:
data.iloc[:, imputer.pattern_log.store_tuple_columns[(1, 1, 1, 1, 1, 1, 1, 1, 1, 'NaN')]].columns

Index(['salary'], dtype='object')

In [93]:
data.tail()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
14994,0.89,0.66,5.0,224.0,2.0,0.0,0.0,0.0,sales,medium
14995,0.67,0.48,6.0,107.0,2.0,1.0,0.0,0.0,hr,low
14996,0.65,0.40,5.0,125.0,4.0,0.0,0.0,0.0,sales,low
14997,0.97,0.58,3.0,200.0,2.0,0.0,0.0,0.0,RandD,medium
14998,0.83,0.57,3.0,137.0,2.0,0.0,0.0,0.0,marketing,medium


### Accessing final models

In [94]:
mdl = imputer.get_model(2) # returns model for pattern 2

In [95]:
mdl.get_feature_name()

'last_evaluation'

In [96]:
mdl.get_model()

[RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
            max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0, warm_start=False)]

### Scaling for preprocessing

In [97]:
from sklearn.preprocessing import StandardScaler

In [98]:
scaler = StandardScaler()

In [99]:
test = data.dropna()

In [100]:
len(test[test.apply(nan_checker.is_nan)])

11259

In [101]:
test.describe()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years
count,11259.000000,11259.000000,11259.000000,11259.000000,11259.000000,11259.000000,11259.000000,11259.000000
mean,0.612595,0.716758,3.804334,201.035794,3.498179,0.143974,0.237144,0.020162
std,0.247815,0.171948,1.231289,50.026808,1.455282,0.351079,0.425350,0.140559
min,0.090000,0.360000,2.000000,96.000000,2.000000,0.000000,0.000000,0.000000
25%,0.440000,0.560000,3.000000,156.000000,3.000000,0.000000,0.000000,0.000000
50%,0.640000,0.720000,4.000000,200.000000,3.000000,0.000000,0.000000,0.000000
75%,0.820000,0.870000,5.000000,245.000000,4.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,7.000000,310.000000,10.000000,1.000000,1.000000,1.000000


In [102]:
test_scaled = scaler.fit_transform(test[test.corr().columns])

In [103]:
test[:5]

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
374,0.97,0.77,3.0,245.0,3.0,0.0,0.0,0.0,hr,medium
749,0.76,0.92,3.0,176.0,2.0,0.0,0.0,0.0,IT,medium
1124,0.82,0.86,5.0,243.0,5.0,0.0,1.0,0.0,IT,low
1499,0.56,0.59,5.0,254.0,4.0,0.0,0.0,0.0,support,high
1874,0.54,0.84,3.0,132.0,3.0,0.0,0.0,0.0,IT,low


In [104]:
pd.DataFrame(test_scaled[:5])

,0,1,2,3,4,5,6,7
0,1.442287,0.309653,-0.653275,0.878852,-0.342340,-0.410108,-0.557551,-0.143445
1,0.594844,1.182049,-0.653275,-0.500470,-1.029523,-0.410108,-0.557551,-0.143445
2,0.836970,0.833091,0.971111,0.838872,1.032025,-0.410108,1.793559,-0.143445
3,-0.212245,-0.737222,0.971111,1.058764,0.344842,-0.410108,-0.557551,-0.143445
4,-0.292954,0.716771,-0.653275,-1.380037,-0.342340,-0.410108,-0.557551,-0.143445


In [105]:
test_rescaled = scaler.inverse_transform(test_scaled)

In [106]:
pd.DataFrame(test_rescaled[:5])

,0,1,2,3,4,5,6,7
0,0.97,0.77,3.0,245.0,3.0,0.0,-2.775558e-17,0.0
1,0.76,0.92,3.0,176.0,2.0,0.0,-2.775558e-17,0.0
2,0.82,0.86,5.0,243.0,5.0,0.0,1.000000e+00,0.0
3,0.56,0.59,5.0,254.0,4.0,0.0,-2.775558e-17,0.0
4,0.54,0.84,3.0,132.0,3.0,0.0,-2.775558e-17,0.0


In [112]:
reload(impyte3)
testdf = pd.read_csv('data/masterdf_201710230_andirs.csv', low_memory=False, encoding='ISO-8859-1')
imp = impyte3.Impyter()
_ = imp.load_data(testdf)
imp.pattern()

,Unnamed: 0,Incident Date,EAS,Incident_Year,Incident_Cat,Incident_Dummy,Neighborhood,Location_y,Address,Building_Cat,Yr_Property_Built,Num_Bathrooms,Num_Bedrooms,Num_Rooms,Num_Stories,Num_Units,Perc_Ownership,Land_Value,Property_Area,Assessed_Improvement_Val,Tot_Rooms,landval_psqft,count potential fire control,count all complaints,count all complaints not corrected,count potential fire control not corrected,count fire emergency safety,count potential fire cause,count fire emergency safety not corrected,count potential fire cause not corrected,next_fire_dpt_address,next_fire_dpt_distance,next_fire_dpt_latlong,Count
0,1,NaN,1,NaN,NaN,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,170517
1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,24767
2,1,1,1,1,NaN,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,24


In [114]:
imp.get_summary()

,Complete,Missing,Percentage,Unique
Incident_Year,24791,170517,87.31 %,16
Incident Date,24791,170517,87.31 %,5205
Incident_Cat,24767,170541,87.32 %,7


In [116]:
imp.get_summary(importance_filter=False)

,Complete,Missing,Percentage,Unique
Unnamed: 0,195308,0,0.00 %,195308
next_fire_dpt_address,195308,0,0.00 %,44
count potential fire cause not corrected,195308,0,0.00 %,5
count fire emergency safety not corrected,195308,0,0.00 %,4
count potential fire cause,195308,0,0.00 %,6
count fire emergency safety,195308,0,0.00 %,11
count potential fire control not corrected,195308,0,0.00 %,6
count all complaints not corrected,195308,0,0.00 %,8
count all complaints,195308,0,0.00 %,18
count potential fire control,195308,0,0.00 %,15


### Timing some of the functions
Below is a piece of information on efficiency of the functions. This doesn't reflect complexity of the functions, but it will show an runtime estimate for your local machine.

In [117]:
%timeit nan_checker.is_nan(["", 'None', 'NaN'])
%timeit nan_checker.is_nan(["", 'None', 'NaN'], nan_vals=['', None, 'None', 'NaN'])

3.9 µs ± 246 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
4.14 µs ± 599 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [126]:
imp.save_model("testmodel.pkl")

## Ensemble
Ensemble iterates over several estimators and imputes accordingly. So far it prints the results for easy comparison.

In [45]:
imp.ensemble(estimator_list=["rf", "dt"])

Scoring Threshold             Classification                Regression                    
                              None                          None                          

Pattern: Label                Score                         Estimator                     
1: sales                      0.332 (f1_macro)              RandomForestClassifier          filled...
2: average_montly_hours       0.324 (r2)                    RandomForestRegressor           filled...
3: number_project             0.571 (f1_macro)              RandomForestClassifier          filled...
4: salary                     0.521 (f1_macro)              RandomForestClassifier          filled...
5: promotion_last_5years      0.741 (f1_macro)              RandomForestClassifier          filled...
6: time_spend_company         0.578 (f1_macro)              RandomForestClassifier          filled...
7: last_evaluation            0.274 (r2)                    RandomForestRegressor           filled...
8: Work_acci